<a href="https://colab.research.google.com/github/pri-nitta/FIAP_IA/blob/main/Redes_Neurais_recorrentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Escrevendo um novo capitulo como Lewis Carroll

In [2]:
import tensorflow as tf

import numpy as np
import os
import time
import pandas as pd

In [3]:
text = open('/content/drive/MyDrive/wonderland.txt', 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')

Length of text: 147683 characters


In [4]:
# Visualizando uma amostragem das 300 primeiras palavras
print(text[:300])

CHAPTER I. Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, 'and what is the use of a book,' thought Al


In [5]:
# Total de caracteres únicos no arquivo - não se repetem
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

70 unique characters


## Vetorização do texto

In [6]:
# Exemplo do uso de tokenização simples com unidecode slplit
example_texts = ['alice', 'coelho']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars


<tf.RaggedTensor [[b'a', b'l', b'i', b'c', b'e'], [b'c', b'o', b'e', b'l', b'h', b'o']]>

In [7]:
# Utilizando o StringLookup - colocar um código/ token para cada palavra
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [8]:
# Analisando os IDs unicos
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[45, 56, 53, 47, 49], [47, 59, 49, 56, 52, 59]]>

In [9]:
# Se eu trazer a opção invert=True, ele traz de volta cada token
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [10]:
# Você pode tf.strings.reduce_join para juntar os caracteres de volta em strings.

tf.strings.reduce_join(chars, axis=-1).numpy()


array([b'alice', b'coelho'], dtype=object)

In [11]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

  #receber os caracteres e fazer a conversão para texto

In [12]:
# Pegando a lista de IDs do texto
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(147683,), dtype=int64, numpy=array([18, 23, 16, ...,  1,  2,  1])>

In [13]:
# Converter o vetor de texto em um fluxo de índices de caracteres.
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [14]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))


C
H
A
P
T
E
R
 
I
.


In [15]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [16]:
# O método batch permite converter facilmente esses caracteres individuais em sequências do tamanho desejado.
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'C' b'H' b'A' b'P' b'T' b'E' b'R' b' ' b'I' b'.' b' ' b'D' b'o' b'w'
 b'n' b' ' b't' b'h' b'e' b' ' b'R' b'a' b'b' b'b' b'i' b't' b'-' b'H'
 b'o' b'l' b'e' b'\r' b'\n' b'\r' b'\n' b'A' b'l' b'i' b'c' b'e' b' ' b'w'
 b'a' b's' b' ' b'b' b'e' b'g' b'i' b'n' b'n' b'i' b'n' b'g' b' ' b't'
 b'o' b' ' b'g' b'e' b't' b' ' b'v' b'e' b'r' b'y' b' ' b't' b'i' b'r'
 b'e' b'd' b' ' b'o' b'f' b' ' b's' b'i' b't' b't' b'i' b'n' b'g' b' '
 b'b' b'y' b' ' b'h' b'e' b'r' b' ' b's' b'i' b's' b't' b'e' b'r' b' '
 b'o' b'n' b' '], shape=(101,), dtype=string)


In [17]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'CHAPTER I. Down the Rabbit-Hole\r\n\r\nAlice was beginning to get very tired of sitting by her sister on '
b'the\r\nbank, and of having nothing to do: once or twice she had peeped into the\r\nbook her sister was re'
b"ading, but it had no pictures or conversations in\r\nit, 'and what is the use of a book,' thought Alice"
b" 'without pictures or\r\nconversations?'\r\n\r\nSo she was considering in her own mind (as well as she coul"
b'd, for the\r\nhot day made her feel very sleepy and stupid), whether the pleasure\r\nof making a daisy-ch'


In [18]:
def split_input_target(sequence):
    input_text = sequence[:-1] # todos os elementos da sequência original, exceto o último
    target_text = sequence[1:] # pega todos os elementos da sequência original, exceto o primeiro
    return input_text, target_text

In [19]:
split_input_target(list("Rainha"))

(['R', 'a', 'i', 'n', 'h'], ['a', 'i', 'n', 'h', 'a'])

In [20]:
dataset = sequences.map(split_input_target)

In [21]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'CHAPTER I. Down the Rabbit-Hole\r\n\r\nAlice was beginning to get very tired of sitting by her sister on'
Target: b'HAPTER I. Down the Rabbit-Hole\r\n\r\nAlice was beginning to get very tired of sitting by her sister on '


# Testes

In [22]:
# Define o tamanho do lote (batch size)
BATCH_SIZE = 64

# Define o tamanho do buffer para embaralhar o dataset
# (O TF data é projetado para trabalhar com sequências possivelmente infinitas,
# então ele não tenta embaralhar a sequência inteira na memória. Em vez disso,
# ele mantém um buffer no qual embaralha os elementos).
BUFFER_SIZE = 10000

# Aplica operações no dataset:
dataset = (
    dataset
    # Embaralha o dataset usando o tamanho do buffer definido
    .shuffle(BUFFER_SIZE)
    # Agrupa os elementos em lotes de tamanho definido (BATCH_SIZE)
    # drop_remainder=True garante que apenas lotes completos sejam mantidos
    .batch(BATCH_SIZE, drop_remainder=True)
    # Pré-carrega os lotes para otimizar a performance de treinamento
    # tf.data.experimental.AUTOTUNE ajusta automaticamente o pré-carregamento
    .prefetch(tf.data.experimental.AUTOTUNE)
)

# Retorna o dataset processado
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

# Construir o modelo de rede neural

In [23]:
# Tamanho do vocabulário
vocab_size = len(vocab)

# Tamanho da dimensão de embedding
embedding_dim = 256

# Número de memórias RNN
rnn_units = 1024

#logit = vai ver a probabilidade do proximo caractere ser o certo.

In [24]:
# Definição da classe do modelo, que herda de tf.keras.Model
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super(MyModel, self).__init__()
    # Camada de embedding que converte IDs de palavras em vetores densos de dimensão embedding_dim
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    # Camada GRU com o número de unidades especificado, retornando sequências completas e estado final
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    # Camada densa que transforma as saídas da GRU em logits para cada palavra no vocabulário
    # As saídas da GRU (Gated Recurrent Unit) em logits referem-se aos valores numéricos não normalizados produzidos pela camada densa após a GRU.
    # Esses valores representam a pontuação ou a "logit" para cada classe do vocabulário, antes de serem passados por uma função de ativação, como softmax,
    # para converter essas pontuações em probabilidades.
    self.dense = tf.keras.layers.Dense(vocab_size)

  # Definição do método call, que especifica como os dados passam pelo modelo
  def call(self, inputs, states=None, return_state=False, training=False):
    # Entrada inicial
    x = inputs
    # Aplicação da camada de embedding às entradas
    x = self.embedding(x, training=training)
    # Se nenhum estado inicial for fornecido, inicializa o estado da GRU
    if states is None:
      states = [tf.zeros((x.shape[0], self.gru.units))]
    # Passa os embeddings pela GRU, obtendo as saídas e o novo estado
    x, states = self.gru(x, initial_state=states, training=training)
    # Passa as saídas da GRU pela camada densa para obter logits
    x = self.dense(x, training=training)

    # Se return_state for True, retorna as saídas e o estado; caso contrário, retorna apenas as saídas
    if return_state:
      return x, states
    else:
      return x

# Instanciação do modelo MyModel com os parâmetros fornecidos
model = MyModel(
    # Certifique-se de que o tamanho do vocabulário corresponde ao das camadas StringLookup
    vocab_size=len(ids_from_chars.get_vocabulary()),  # Tamanho do vocabulário
    embedding_dim=embedding_dim,                      # Dimensão dos embeddings
    rnn_units=rnn_units)                              # Número de unidades da GRU

In [25]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 71) # (batch_size, sequence_length, vocab_size)


In [26]:
model.summary()

Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (64, 100, 256)              │          18,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ((64, 100, 1024), (64,      │       3,938,304 │
│                                      │ 1024))                      │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (64, 100, 71)               │          72,775 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,029,255 (15.37 MB)

 Trainable params: 4,029,255 (15.37 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [28]:
sampled_indices

array([45, 36, 65, 27, 70, 10, 37, 68,  0, 57, 43, 67, 22, 48,  3, 54, 16,
        3, 60, 36, 19, 45,  9, 19, 54, 21, 32, 15, 40,  4, 40, 54, 56, 42,
       35,  2, 66, 17, 41, 34, 10, 27, 52, 44, 60, 38, 68, 65, 58,  9, 28,
       38, 17, 35, 69, 35, 49,  5, 58,  2, 30, 45, 14,  6, 48, 25, 33, 56,
       57, 42, 37, 66, 51, 33, 62, 52, 42, 55, 66, 53, 29, 27, 18, 22, 55,
       13, 20, 54, 27, 30, 26, 30,  1, 59, 55, 65, 57, 13, 27, 50])

In [29]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'toffee, and hot\r\nbuttered toast,) she very soon finished it off.\r\n\r\n  *    *    *    *    *    *    '

Next Char Predictions:
 b'aUuLz,Vx[UNK]m]wGd jA pUDa*DjFQ?Y!Yjl[T\rvBZS,Lh_pWxun*MWBTyTe"n\rOa;\'dJRlm[VvgRrh[kviNLCGk:EjLOKO\nokum:Lf'


# Treinamento

In [30]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
#pegar as probabilidades e utilizando multiclasses

In [31]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 71)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.262016, shape=(), dtype=float32)


In [32]:
tf.exp(example_batch_mean_loss).numpy()

70.952866

In [33]:
model.compile(optimizer='adam', loss=loss)

In [34]:
# Diretório onde o checkpoint será salvo
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [35]:
EPOCHS = 20

In [36]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 133s 6s/step - loss: 4.2889
Epoch 2/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 137s 6s/step - loss: 2.8433
Epoch 3/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 133s 6s/step - loss: 2.4620
Epoch 4/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 139s 6s/step - loss: 2.2758
Epoch 5/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 127s 6s/step - loss: 2.1346
Epoch 6/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 126s 6s/step - loss: 2.0251
Epoch 7/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 137s 6s/step - loss: 1.9273
Epoch 8/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 130s 6s/step - loss: 1.8305
Epoch 9/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 124s 6s/step - loss: 1.7487
Epoch 10/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 143s 6s/step - loss: 1.6686
Epoch 11/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 123s 6s/step - loss: 1.6042
Epoch 12/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 121s 5s/step - loss: 1.5383
Epoch 13/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 121s 5s/step - loss: 1.4597
Epoch 14/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 141s 5s/step - loss: 1.4057
Epoch 15/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 142s 5s/step - loss: 1.3553
Epoc

In [37]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    # Chama o construtor da classe base
    super().__init__()
    # Define a temperatura para ajuste da aleatoriedade da geração
    # temperature: um parâmetro opcional para ajustar a aleatoriedade da geração (padrão é 1.0)
    # quanto maior a temperatura maior a criatividade/ alucinação
    self.temperature = temperature
    # Define o modelo subjacente a ser usado para previsão
    self.model = model
    # Mapeamento de IDs para caracteres
    self.chars_from_ids = chars_from_ids
    # Mapeamento de caracteres para IDs
    self.ids_from_chars = ids_from_chars

    # Cria uma máscara para impedir a geração do token "[UNK]"
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Coloca -inf em cada índice a ser evitado
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Ajusta a forma ao vocabulário
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    # Converte a máscara esparsa em uma densa
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Converte strings em IDs de tokens
    # chars_from_ids: uma função que mapeia IDs para caracteres
    # ids_from_chars: uma função que mapeia caracteres para IDs
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Executa o modelo
    # predicted_logits.shape é [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Usa apenas a última previsão
    predicted_logits = predicted_logits[:, -1, :]
    # Ajusta os logits pela temperatura
    predicted_logits = predicted_logits/self.temperature
    # Aplica a máscara de previsão para evitar a geração de "[UNK]"
    predicted_logits = predicted_logits + self.prediction_mask

    # Amostra os logits de saída para gerar IDs de tokens
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Converte de IDs de tokens para caracteres
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Retorna os caracteres e o estado do modelo
    return predicted_chars, states

# Exemplo de instância da classe OneStep:
# model: um modelo de previsão de texto (por exemplo, um modelo GRU ou LSTM treinado)

#
# one_step_model = OneStep(model, chars_from_ids, ids_from_chars, temperature=1.0)

In [38]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [39]:
start = time.time() # Inicia o cronômetro para medir o tempo de execução
states = None # Inicializa o estado da RNN como None

next_char = tf.constant(['Alice:']) # Define a string inicial para a geração de texto

result = [next_char] # Cria uma lista para armazenar os caracteres gerados

# Loop para gerar 1000 caracteres
for n in range(1000):
  # Gera o próximo caractere e o novo estado do modelo
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  # Adiciona o caractere gerado à lista de resultados
  result.append(next_char)

# Junta todos os caracteres gerados em uma única string
result = tf.strings.join(result)
# Para o cronômetro e calcula o tempo de execução
end = time.time()

# Imprime o texto gerado
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

# Imprime o tempo de execução
print('\nRun time:', end - start)

Alice: (here when she was a pentried
quite a surrily next quite about it, how the Gryphoon on tire when ever spoke.

However, out when the Duchess seemed to raim nothing, half long live a tho wime she callled out that she did slace-began.

The Doghos out to see. Picause!
 Which said, with one, even by the Mock Turtle, seching he
the mome first were shrieed woblain the morel. 'Bale one's it exturce is, that'se whitine plas
to have the nexp! Of cardle, your Majesty,' said Alice.

'Will the tunn't eagerly, of 'Wust,' said the other, now the shouted to spee_ she gound tho
removed it her upeapitt it wasn't yet trit the QUeen, it said and went on sitter.

Alice thought, pushing a vury like thist!' wenteded the raote, when seem to have just
as she call in a dance of the cauers. It was the Hatter was sighing it, and tull the next
day the beauth it hard: the Mouse shreled this
miguted on its mean remarked, 'What do showing
the Mock Turt ears touse of pelf. I should thing,' she she he 

________